In [ ]:
# !pip install langchain_community langchainhub chromadb langchain langchain-openai

In [ ]:
!pip install langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://www.educosys.com/course/genai"])

docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Learn, Build, Deploy and Apply Generative AI', 'language': 'en'}, page_content="Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 6th Feb, 2026Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · Fri, Sat and Sun · 9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Architectures: Feedforward, RNN, and CNN Mini Project - Build a Simple Neural Network Using TensorFlow Mi

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
splits = text_splitter.split_documents(docs)

In [ ]:
print(splits[0])
print(splits[1])
print(splits[2])

page_content='Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 6th Feb, 2026Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · Fri, Sat and Sun · 9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week' metadata={'source': 'https://www.educosys.com/course/genai', 'title': 'Hands-on Generative AI Course', 'description': 'Learn, Build, Deploy and Apply Generative AI', 'language': 'en'}
page_content='RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations for AI Probability, Statistics, and Linear Algebra Basics of Neural Networks Gradient Descent and Optimization Basics Arc

In [ ]:
print(len(splits))

39


In [ ]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

# vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

/tmp/ipython-input-2599906244.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
print(vectorstore._collection.count())

39


In [ ]:
print(vectorstore._collection.get())

{'ids': ['495a3eca-e57d-4321-8eff-c4d2d1acbdc3', 'de7899e5-0ce9-416a-88ea-eec2602b74f3', 'e8a3566f-81cf-4fd1-9295-5bd9c8c78d0e', 'a926d3da-8c8d-45cb-9a10-8422fad7d37f', '396bfa2e-4c0d-4622-b0b1-761996e0ec03', 'c3e08a7a-b2d2-42a7-88df-5adff0134dab', '5d789c95-21c5-47f1-9bbd-7c3ed66d7b81', '372442f8-ae4b-43d4-a9c6-2b325b570efe', '36fa13ad-54ef-4727-b6cb-98c57d4cfe77', '6e066847-dd91-429d-91be-e43e5828632e', '2055fa7c-d94f-469a-ba63-36158ba4d1b9', 'f6e302c4-44f3-40c8-9423-1bde831e6d3f', 'e5612ae6-43f3-4eb3-863b-2e24c31ef1c7', '404d206d-b53d-4752-81ef-04a9c47e98c0', 'd4141d98-811b-4789-8281-925c322fe044', '8056ba5c-698f-4c7e-a47f-dee8443b21f9', '04b0830f-155e-46d6-8a71-7a24cf3ad7b1', 'df3baa85-fbbf-40c0-a80b-ad51cc9a66e0', '1ea3b64d-009a-4e83-b185-2e4f2e65fc59', '0f0c7ffe-cd20-43f0-94bd-d4030459e752', '86f1a369-17a3-4da8-b97b-ddc0776a8234', 'fc0e738c-f48a-4011-87ee-0fec9f82caba', '1b5c601a-05d6-41f7-bcfa-f350e1316482', 'b94a7d30-de37-4b67-93be-4735d6c37b3e', 'b441a714-0204-4155-a727-ad6232

In [ ]:
print("\nCollection 1 - ", vectorstore._collection.get(ids=['28651d9a-ab51-41f8-ab83-e68285623c4e'], include=["embeddings", "documents"]))
print("\nCollection 2 - ", vectorstore._collection.get(ids=['054dee19-19ed-4574-bc51-511060fd707a'], include=["embeddings", "documents"]))
print("\nCollection 3 - ", vectorstore._collection.get(ids=['2fd71cb4-835a-43c5-b920-b7e1be51c450'], include=["embeddings", "documents"]))


Collection 1 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 2 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}

Collection 3 -  {'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'included': ['embeddings', 'documents'], 'data': None, 'metadatas': None}


In [ ]:
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

In [ ]:
# from langchain import hub
# prompt = hub.pull("rlm/rag-prompt")

In [ ]:
# The 'hub' object (for pulling prompts from LangChain Hub) is imported directly from the 'langchain_hub' package.
# However, it appears there's still an issue importing 'langchain_hub' or accessing 'hub.pull()'.
# To ensure the prompt is defined and allow the chain to run, we will explicitly define the common 'rlm/rag-prompt' template.
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."),
    ("human", "Question: {question}\nContext: {context}\nAnswer:")
])

In [ ]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI()

In [ ]:
# from langchain_community.llms import HuggingFaceHub
# llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0.5, "max_length":512}, task="text2text-generation")

from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-970550028.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [ ]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'Lifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week'

In [ ]:
rag_chain.invoke("Are the testimonials for the course available? Name the studenst who have shared testimonials")

'Keerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.'

In [ ]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes.'

In [ ]:
rag_chain.invoke("Are the certificates for the course provided?")

'Yes.'

In [ ]:
rag_chain.invoke("What all projects are covered in the course?")

'Build, Deploy and Apply Generative AI7 weeks  Fri, Sat and Sun  9:00 PM - 11:00 PM IST (every class) + Post-class doubt supportLast fully LIVE batch. All future courses will follow a hybrid format!Access all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week Software EngineerOptum-United HealthGroupKeerti explains everything in such simple and creative manner, even difficult and huge topics became easy to understand.'

In [ ]:
from langchain_core.runnables import RunnableLambda

In [ ]:
# def print_prompt(prompt_text):
#   print("Prompt - ", prompt_text)
#   return prompt_text

In [ ]:
# rag_chain_with_print = ({"context" : retriever | format_docs, "question" : RunnablePassthrough()}
#              | prompt
#              | RunnableLambda(print_prompt)
#              | llm
#              | StrOutputParser())

In [ ]:
# rag_chain_with_print.invoke("What all projects are covered in the course?")

In [ ]:
## Testing of openai api key

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

# Create client (API key is read from environment variable)
client = OpenAI()

print("🤖 Simple Chatbot — type 'exit' to quit\n")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye! 👋")
        break

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a friendly chatbot that helps users."},
            {"role": "user", "content": user_input}
        ]
    )

    reply = response.choices[0].message.content
    print(f"Chatbot: {reply}\n")


🤖 Simple Chatbot — type 'exit' to quit

You: hi
Chatbot: Hello! How can I assist you today?

You: what are some of the latest news
Chatbot: I'm unable to provide real-time news updates, but I can summarize major events and trends up to October 2023. If you're looking for the latest information, I recommend checking reputable news websites or apps. Is there a specific topic or area you're interested in? I can help summarize recent trends or events in that area!

You: which are some of the free tier gpt models?
Chatbot: As of October 2023, some platforms offer free tier access to various GPT models. Here are a few options where you might find free access to GPT-like models:

1. **OpenAI's ChatGPT**: OpenAI offers a free tier for users to access its models on the ChatGPT platform. While there are paid options (such as ChatGPT Plus), users can still use a basic version for free, typically with some usage limits.

2. **Hugging Face**: Hugging Face provides a wide range of language models, i

KeyboardInterrupt: Interrupted by user